# Credits

See main readme for credits.

# Dependancies and supporting functions
Loading dependancies and supporting functions by running the code block below.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
%matplotlib nbagg
%matplotlib inline

In [ ]:
def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if torch.cuda.is_available():
        return x.cuda()
    return x

def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if torch.cuda.is_available():
        return x.cpu().data.numpy()
    return x.data.numpy()

# Auto-encoders 101
In this notebook you will implement a simple auto-encoder (AE). We assume that you are already familiar with the basics of neural networks. We'll start by defining an AE similar to the one used for the finetuning step by [Geoffrey Hinton and Ruslan Salakhutdinov](https://www.cs.toronto.edu/~hinton/science.pdf). We'll experiment with the AE setup and try to run it on the MNIST dataset. There has been a wide variety of research into the field of auto-encoders and the technique that you're about to learn is very simple compared to recent advances (e.g. [the Ladder network](https://arxiv.org/abs/1507.02672) and [Variational AEs](https://arxiv.org/abs/1312.6114)). However, the basic idea stays the same.

AEs are used within unsupervised learning, in which you do not have a target $y$. Instead it *encodes* an input $x$ into a latent state $z$ and decodes $z$ into a reconstruction $\hat{x}$. This way the parameters of the network can be optimized w.r.t. the difference between $x$ and $\hat{x}$. Depending on the input distribution, the difference can be measured in various ways, e.g. mean squared error (MSE). In many applications the auto-encoder will find an internal state of each data point corresponding to a feature. So if we are to model the MNIST dataset, one could expect that the internal state would correspond to a digit-class and/or the shape.

*The exercises are found at the bottom of the notebook*

## MNIST
First let us load the MNIST dataset and plot a few examples. We only load a limited amount of classes to speed up training.

In [ ]:
!if [ ! -f mnist.npz ]; then wget -N https://www.dropbox.com/s/qxywaq7nx19z72p/mnist.npz; else echo "mnist.npz already downloaded"; fi

In [ ]:
from sklearn.utils import shuffle

# To speed up training we'll only work on a subset of the data containing only the numbers 0, 1.
data = np.load('mnist.npz')
num_classes = 2
idxs_train = []
idxs_valid = []
idxs_test = []
for i in range(num_classes):
    idxs_train += np.where(data['y_train'] == i)[0].tolist()
    idxs_valid += np.where(data['y_valid'] == i)[0].tolist()
    idxs_test += np.where(data['y_test'] == i)[0].tolist()

x_train = data['X_train'][idxs_train].astype('float32')
# Since this is unsupervised, the targets are only used for validation.
targets_train = data['y_train'][idxs_train].astype('int32')
x_train, targets_train = shuffle(x_train, targets_train, random_state=1234)


x_valid = data['X_valid'][idxs_valid].astype('float32')
targets_valid = data['y_valid'][idxs_valid].astype('int32')

x_test = data['X_test'][idxs_test].astype('float32')
targets_test = data['y_test'][idxs_test].astype('int32')

print("training set dim(%i, %i)." % x_train.shape)
print("validation set dim(%i, %i)." % x_valid.shape)
print("test set dim(%i, %i)." % x_test.shape)

In [ ]:
#plot a few MNIST examples
idx = 0
canvas = np.zeros((28*10, 10*28))
for i in range(10):
    for j in range(10):
        canvas[i*28:(i+1)*28, j*28:(j+1)*28] = x_train[idx].reshape((28, 28))
        idx += 1
plt.figure(figsize=(7, 7))
plt.axis('off')
plt.imshow(canvas, cmap='gray')
plt.title('MNIST handwritten digits')

### Building the model
When defining the model the latent layer $z$ must act as a bottleneck of information. We initialize the AE with 1 hidden layer in the encoder and decoder using ReLU units as nonlinearities. The latent layer has a dimensionality of 2 in order to make it easy to visualise. Since $x$ are pixel intensities that are normalized between 0 and 1, we use the sigmoid nonlinearity to model the reconstruction.

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

from torch.nn import Linear
from torch.nn.functional import relu, sigmoid

In [ ]:
# define size variables
num_features = x_train.shape[1]
l_enc_features = 128
l_z_features = 2
l_dec_features = 128

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # encoder layer
        self.l_enc = Linear(in_features=num_features,
                            out_features=l_enc_features)
        # bottleneck layer
        self.l_z = Linear(in_features=l_enc_features,
                          out_features=l_z_features)
        # decoder layer
        self.l_dec = Linear(in_features=l_z_features,
                            out_features=l_dec_features)
        # output layer, projecting back to image size
        self.l_out = Linear(in_features=l_dec_features,
                            out_features=num_features)

    def forward(self, x): 
        outputs = {}
        x = relu(self.l_enc(x))
        # we don't apply an activation to the bottleneck layer; self.l_z
        x = self.l_z(x)
        outputs['l_z'] = x
        x = relu(self.l_dec(x))
        # apply sigmoid to output to get pixel intensities between 0 and 1
        x = sigmoid(self.l_out(x))
        outputs['l_out'] = x
        return outputs


net = Net()
if torch.cuda.is_available():
    net.cuda()
print(net)

Following we define the PyTorch functions for training and evaluation.

In [ ]:
def mse_pixel_loss(y, t):
    """ Mean Squared Error (MSE)
    
    Parameters
    ----------
    y: torch.autograd.Variable
    t: torch.autograd.Variable
    
    Returns
    -------
    torch.autograd.Variable
    """
    loss_per_pixel = torch.pow(t.sub(y), exponent=2)
    return torch.mean(loss_per_pixel)

# if you want L2 regularization, then add weight_decay to SGD
optimizer = optim.SGD(net.parameters(), lr=0.25)

loss_function = mse_pixel_loss

This function can be called with `torch.Tensor`'s and will return a float, but we will use `Variable` as inputs to compute and use gradients.

In [ ]:
# create two random tensors of size height 1, and width 3
_y, _t = torch.randn(1, 3), torch.randn(1, 3)

# apply mean squared error by comparing the two tensors
x = mse_pixel_loss(_y, _t)
# returns a float
print(type(x))
print(x)

In [ ]:
# use same two tensors, but wrap them in a Variable
# notice, that we put requires_grad=True on the prediction (__y),
# i.e. when .backward() is applied, the gradients are computed and stored on __y
__y = Variable(_y, requires_grad=True)
# no need to store grad for target Variable
__t = Variable(_t)

# apply mean squared error by comparing the two Variables
x = mse_pixel_loss(__y, __t)
# returns Variable containing torch.FloatTensor of size 1
print(type(x))
print(x)

# the Variable should not have any grad before applying .backward()
print(__y.grad)
x.backward()
print(__y.grad)

In [ ]:
# test the forward pass
_x_test = np.zeros(shape=(32, num_features), dtype='float32')
# expect output size of [32, num_features]
print(num_features)
out_dict = net(get_variable(Variable(torch.from_numpy(_x_test))))
print(out_dict['l_out'].size())

In the training loop we sample each batch and evaluate the error, latent space, and reconstructions on every epoch.

**NOTE** this will take a while on CPU.

In [ ]:
batch_size = 100
num_epochs = 100
num_samples_train = x_train.shape[0]
num_batches_train = num_samples_train // batch_size
num_samples_valid = x_valid.shape[0]
num_batches_valid = num_samples_valid // batch_size
updates = []
tmp_img = "tmp_ae_out.png"

train_loss = []
valid_loss = []
cur_loss = 0
plt.figure(figsize=(12, 24))

for epoch in range(num_epochs):
    cur_loss = []
    net.train()
    for i in range(num_batches_train):
        idxs = np.random.choice(range(x_train.shape[0]), size=(batch_size), replace=False)    
        x_batch = x_train[idxs]
        _x = get_variable(Variable(torch.from_numpy(x_batch)))
        out_dict = net(_x)
        # note, target is the original tensor, as we're working with auto-encoders
        loss = loss_function(out_dict['l_out'], _x)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        cur_loss.append(get_numpy(loss))
    train_loss.append(np.mean(cur_loss))
    updates.append(batch_size * num_batches_train * (epoch + 1))

    # evaluate
    net.eval()
    _target = get_variable(Variable(torch.from_numpy(x_valid)))
    out_dict = net(_target)
    loss = loss_function(out_dict['l_out'], _target)
    
    # used later
    eval_out = get_numpy(out_dict['l_out'])
    eval_z = get_numpy(out_dict['l_z'])
    
    valid_loss.append(get_numpy(loss))
    
    if epoch == 0:
        continue

    # Plotting
    plt.subplot(num_classes + 1, 2, 1)
    plt.title('Error')
    plt.legend(['Train Error', 'Valid Error'])
    plt.xlabel('Updates'), plt.ylabel('Error')
    plt.plot(updates, train_loss, color="black")
    plt.plot(updates, valid_loss, color="grey")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.grid('on')

    plt.subplot(num_classes+1,2,2)
    plt.cla()
    plt.title('Latent space')
    plt.xlabel('z0'), plt.ylabel('z1')
    color = iter(plt.get_cmap('brg')(np.linspace(0, 1.0, num_classes)))
    for i in range(num_classes):
        clr = next(color)
        plt.scatter(eval_z[targets_valid==i, 0], eval_z[targets_valid==i, 1], c=clr, s=5., lw=0, marker='o', )
    plt.grid('on')

    c=0
    for k in range(3, 3 + num_classes * 2, 2):
        plt.subplot(num_classes + 1, 2, k)
        plt.cla()
        plt.title('Inputs for %i' % c)
        plt.axis('off')
        idx = 0
        canvas = np.zeros((28*10, 10*28))
        for i in range(10):
            for j in range(10):
                canvas[i*28:(i+1)*28, j*28:(j+1)*28] = x_valid[targets_valid==c][idx].reshape((28, 28))
                idx += 1
        plt.imshow(canvas, cmap='gray')

        plt.subplot(num_classes+1,2,k+1)
        plt.cla()
        plt.title('Reconstructions for %i' % c)
        plt.axis('off')
        idx = 0
        canvas = np.zeros((28*10, 10*28))
        for i in range(10):
            for j in range(10):
                canvas[i*28:(i+1)*28, j*28:(j+1)*28] = eval_out[targets_valid==c][idx].reshape((28, 28))
                idx += 1
        plt.imshow(canvas, cmap='gray')
        c+=1

    plt.savefig(tmp_img)
    display(Image(filename=tmp_img))
    clear_output(wait=True)
    break
import os
os.remove(tmp_img)

# Assignments

## Assignment 1 - Analyzing the AE
1. The above implementation of an AE is very simple.
    - Experiment with the number of layers and non-linearities in order to improve the reconstructions
    - What happens with the network when we change the non-linearities in the latent layer (e.g. sigmoid)?
    - Try to increase the number of digit classes in the training set and analyze the results
    - Test different optimization algorithms and decide whether you should use regularizers
       
2. Currently we optimize w.r.t. mean squared error. 
    - Find another error function that could fit this problem better
    - Evaluate whether the error function is a better choice and explain your findings

3. Complexity of the bottleneck.
    - Increase the number of units in the latent layer and train
    - Visualize by using [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) or [t-SNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html)

## Assignment 2 - Adding classification

The above training has been performed unsupervised. Now let us assume that we only have a fraction of labeled data points from each class (implemented below). As we know, semi-supervised learning can be utilized by combining unsupervised and supervised learning. Now you must analyze whether a trained AE from the above exercise can aid a classifier.

1. Build a simple classifier (like the ones from week1) where you:
    - Train on the labeled dataset and evaluate the results
2. Build a second classifier and train on the latent output $z$ of the AE.
3. Build a third classifier and train on the reconstructions of the AE.
4. Evaluate the classifiers against each other and implement a model that improves the classification by combining the input, latent output, and reconstruction.

In [ ]:
# Generate a subset of labeled data points

num_labeled = 10 # You decide on the size of the fraction...

def onehot(t, num_classes):
    out = np.zeros((t.shape[0], num_classes))
    for row, col in enumerate(t):
        out[row, col] = 1
    return out

idxs_train_l = []
for i in range(num_classes):
    idxs = np.where(targets_train == i)[0]
    idxs_train_l += np.random.choice(idxs, size=num_labeled).tolist()

x_train_l = x_train[idxs_train_l]
targets_train_l = targets_train[idxs_train_l]
print("labeled training set dim(%i, %i)." % x_train_l.shape)

plt.figure(figsize=(12, 7))
for i in range(num_classes*num_labeled):
    im = x_train_l[i].reshape((28, 28))
    plt.subplot(1, num_classes*num_labeled, i + 1)
    plt.imshow(im, cmap='gray')
    plt.axis('off')